In [ ]:
-- the tables to be joined are 
-- driver_details.csv  (DRIVER_DETAILS)
-- driver_standings.csv (DRIVER_STANDINGS)
-- pitstops.csv (PITSTOPS)
-- practices.csv (PRACTICES)
-- sprint_results.csv (SPRINT_RESULTS)

create or replace table driver_table
as
WITH median_time AS (
    SELECT 
        PERCENTILE_CONT(0.5) WITHIN GROUP (
            ORDER BY CASE 
                WHEN POSITION(':' IN TIME) > 0 THEN 
                    SPLIT_PART(TIME, ':', 1)::INT * 60 + SPLIT_PART(TIME, ':', 2)::FLOAT
                ELSE 
                    TIME::FLOAT 
            END
        ) AS median_value
    FROM pitstops
    WHERE TIME IS NOT NULL
)
SELECT DISTINCT 
    a.CAR,
    C.DETAIL,
    a.DRIVER,
    B.DRIVERCODE,
    D.GAP,
    a.GRAND_PRIX,
    C.LAP,
    B.NATIONALITY,
    A.RACE_POSITION,
    C.STOPS,
    CASE 
        WHEN POSITION(':' IN C.TIME) > 0 THEN 
            SPLIT_PART(COALESCE(C.TIME, TO_CHAR(median_time.median_value)), ':', 1)::INT * 60 + SPLIT_PART(C.TIME, ':', 2)::FLOAT
        ELSE 
            COALESCE(C.TIME, TO_CHAR(median_time.median_value))::FLOAT 
    END AS TIME,
    A.YEAR
FROM DRIVER_DETAILS a
LEFT JOIN driver_standings b 
    ON a.driver = b.driver AND a.year = b.year AND a.car = b.car
LEFT JOIN pitstops c 
    ON a.driver = c.driver AND a.year = c.year AND a.car = c.car AND a.grand_prix = c.grand_prix
LEFT JOIN practices d 
    ON a.driver = d.driver AND a.car = d.car AND a.year = d.year AND a.grand_prix = d.grand_prix AND a.race_position = d.pos
LEFT JOIN sprint_results e 
    ON a.driver = e.driver AND a.year = e.year AND a.race_position = e.pos AND a.car = e.car AND a.grand_prix = e.grand_prix
LEFT JOIN race_details f
    ON a.driver = f.driver AND a.year = f.year AND a.race_position = f.pos AND a.car = f.car AND a.grand_prix = f.grand_prix
CROSS JOIN median_time; -- Use CROSS JOIN for scalar CTE results

ALTER TABLE driver_table
ADD CONSTRAINT pk_driver_table PRIMARY KEY (DRIVER, YEAR, GRAND_PRIX);

ALTER TABLE driver_table
ADD CONSTRAINT fk_driver_table
FOREIGN KEY (DRIVER,YEAR,GRAND_PRIX)
REFERENCES driver_table (DRIVER,YEAR,GRAND_PRIX);




select * from driver_table


In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_driver_table AS
WITH median_time AS (
    SELECT 
        MEDIAN(CASE 
            WHEN POSITION(':' IN TIME) > 0 THEN 
                SPLIT_PART(TIME, ':', 1)::INT * 60 + SPLIT_PART(TIME, ':', 2)::FLOAT
            ELSE 
                TIME::FLOAT 
        END) AS median_value
    FROM pitstops
    WHERE TIME IS NOT NULL
),
most_frequent_values AS (
    SELECT 
        (SELECT CAR FROM DRIVER_DETAILS GROUP BY CAR ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_car,
        (SELECT DETAIL FROM pitstops GROUP BY DETAIL ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_detail,
        (SELECT NATIONALITY FROM driver_standings GROUP BY NATIONALITY ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_nationality,
        (SELECT GAP FROM practices GROUP BY GAP ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_gap,
        MEDIAN(LAP) AS median_lap,
        MEDIAN(STOPS) AS median_stops
    FROM pitstops
)
SELECT DISTINCT 
    COALESCE(a.CAR,'unknown') AS CAR,
    COALESCE(C.DETAIL,'unknown') AS DETAIL,
    COALESCE(a.DRIVER, 'unknown') AS DRIVER,
    COALESCE(B.DRIVERCODE, 'unknown') AS DRIVERCODE,
    COALESCE(D.GAP,'unknown') AS GAP,
    COALESCE(a.GRAND_PRIX, 'unknown') AS GRAND_PRIX,
    COALESCE(C.LAP, -999) AS LAP,
    COALESCE(B.NATIONALITY,'unknown') AS NATIONALITY,
    -- Replace NULL RACE_POSITION with 'unknown'
    COALESCE(A.RACE_POSITION, 'unknown') AS RACE_POSITION,
    COALESCE(C.STOPS,-999) AS STOPS,
    COALESCE(C.Time,'unknown') AS TIME,
    COALESCE(A.YEAR, -999) AS YEAR
FROM DRIVER_DETAILS a
LEFT JOIN driver_standings B 
    ON a.driver = b.driver AND a.year = b.year AND a.car = b.car
LEFT JOIN pitstops C
    ON a.driver = c.driver AND a.year = c.year AND a.car = c.car AND a.grand_prix = c.grand_prix
LEFT JOIN practices D
    ON a.driver = d.driver AND a.car = d.car AND a.year = d.year AND a.grand_prix = d.grand_prix AND a.race_position = d.pos
LEFT JOIN sprint_results E
    ON a.driver = e.driver AND a.year = e.year AND a.race_position = e.pos AND a.car = e.car AND a.grand_prix = e.grand_prix,
most_frequent_values, median_time;


SELECT * FROM temp_driver_table;

SELECT *
FROM temp_driver_table
WHERE 
TIME = 'unknown'